#Задание 1

In [1]:
!pip install Cython numpy
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-n5zoo2p4
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-n5zoo2p4
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464624 sha256=c8aeb287150b0bcc0ec6b3b22cd034addb3ca4351e52e6b000161fae726c90d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-sobjynp0/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 7.8MB/s 
     |████████████████████████████████| 378kB 46.9MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=843773 sha256=9715d480b84b5ef2b4863014d0484698e5d3e6fdca704cb3c32f532b035b0f82
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [5]:
from google.colab import files
uploaded = files.upload()

Saving paraphrases.xml to paraphrases.xml


In [0]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [6]:
!adagram-train paraphrases.xml out1.pkl

[INFO] 2020-03-12 10:14:57,856 Building dictionary...
[INFO] 2020-03-12 10:14:58,230 Done! 1811 words.
[INFO] 2020-03-12 10:15:02,666 46.21% -3.9942 0.0134 1.7/3.0 14.66 kwords/sec
[INFO] 2020-03-12 10:15:07,017 92.41% -3.9494 0.0019 2.1/4.0 14.71 kwords/sec
[INFO] 2020-03-12 10:15:07,761 100.00% -3.9499 0.0000 2.5/4.0 14.11 kwords/sec


In [0]:
vm = adagram.VectorModel.load("out1.pkl")

In [0]:
def get_words_in_context(words, window=3):
  words_in_context = []
  word_and_context = []
  #words = words.split(' ')
  for i, word in enumerate(words):
    word_and_context = []
    word_and_context.append(word)
    if i - window >= 0 and i + window + 1 <= len(words):
      context = words[i - window:i] + words[i+1:i + window + 1]
    elif i - window >= 0 and i + window + 1 > len(words):
      context = words[i - window:i] + words[i+1:]
    elif i - window < 0 and i + window + 1 <= len(words):
      context = words[0:i] + words[i+1:i + window + 1]
    else:
      context = words[0:i] + words[i+1:]
    word_and_context.append(context)
    words_in_context.append(word_and_context)




  return words_in_context

In [0]:
def get_embedding_adagram(text, model, window, dim):
    
    word2context = get_words_in_context(text, window)
    #print(word2context)
    #words = Counter(text)
    #total = len(text)
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        try:
          n = model.disambiguate(word, context).argmax()
          vectors[i] = model.sense_vector(word,n) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold

In [0]:
dim = 50
window = 3
X_text_1 = np.zeros((len(data['text_1']), dim))
X_text_2 = np.zeros((len(data['text_2']), dim))

for i, text in enumerate(data['text_1'].values):
    X_text_1[i] = get_embedding_adagram(text.split(' '), vm, window, dim)
    
for i, text in enumerate(data['text_2'].values):
    X_text_2[i] = get_embedding_adagram(text.split(' '), vm, window, dim)

In [0]:
X_text = np.concatenate([X_text_1, X_text_2], axis=1)
y = data['label'].values

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_text, y, cv=5, scoring = 'f1_macro')
scores.mean()

0.16924954078611235

Качество совсем не очень. Возможно, стоило применить нормализацию, но я не был уверен, что адаграм, обученный на 'сырых' текстах, справится с тектсами нормализованными. 

#Задание 2

In [22]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def lesk(word, sentence):
  bestsense = 0
  maxoverlap = 0
  synsets = wn.synsets(word)
  overlaps = {}
  overlaps_with_index = {}
    
  for i, syns in enumerate(synsets):
    definit = set(syns.definition().split(' '))
    sent = set(sentence)
    overlaps[i] = sent.intersection(definit)


  overlaps_len = [(key, len(value)) for key, value in overlaps.items()]
  score = 0
  for key, value in overlaps_len:
    #print('key = ', key)
    #print('value =', value)
    if value > score:
      score = value
      bestsence = key
    else:
      continue

  return bestsence

In [24]:
from google.colab import files
uploaded = files.upload()

Saving corpus_wsd_50k.txt to corpus_wsd_50k.txt


In [0]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [27]:
t = 0
f = 0
for sent in corpus_wsd:
  try:
    ambig_words = []
    ambig_word_tags = []
    sentence = []
    for word_with_oth in sent:
      sentence.append(word_with_oth[2])
      #print(sentence)
    for word_with_oth in sent:
      if word_with_oth[0]:
        ambig_word_tag = word_with_oth[0]
        ambig_word = word_with_oth[2]
        if wn.synsets(ambig_word)[lesk(ambig_word, sentence)] == wn.lemma_from_key(ambig_word_tag).synset():
          t += 1
          #print('true')
        else:
          f += 1
          #print('false')
  except:
    continue    


accuracy = t / (t + f)

print('accuracy = ', accuracy)

accuracy =  0.2804867925154821


Точность плоховата, но я, честно сказать, думал, что будет ещё хуже.